In [ ]:
from dotenv import load_dotenv
from anthropic import Anthropic

import os

NO_MORE_CODE = "No more improvements"

load_dotenv(os.path.abspath(os.path.join(os.getcwd(), "../.env")))

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))


def get_messages(prompt, system_prompt="") -> str:
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        messages=prompt,
        max_tokens=1024 * 1,
        temperature=0,
        system=system_prompt
    )

    return response


In [ ]:
import re


def write_self_refine(message: str):
    code = get_init(message)
    revision_num = 1
    while True:
        feedback = get_feedback(code)
        if (is_stop_conditions(feedback, revision_num)):
            break

        print(f"code {revision_num} : {code}")
        print(f"feedback {revision_num} : {feedback}")
        code = refine_code(code, feedback)
        revision_num += 1
    return code


def get_init(message: str) -> str:
    prompt = """
    당신은 파이썬 프로그래머입니다.
    당신은 주어진 문제를 해결하기 위해, 파이썬 코드를 작성해야 합니다.
    코드 본문은 <code> 태그 내에 작성해주세요.
    """

    response = get_messages(message, prompt)
    return extract_code(response.content[0].text)


def extract_code(text) -> str:
    code_match = re.search(r"<code>\s*(.*?)\s*</code>", text, re.DOTALL)

    if code_match:
        return code_match.group(1).strip()

    raise ValueError("No code found")


def get_feedback(code: str) -> str:
    messages = [
        {"role": "user", "content": f"""
        <code> 태그 내의 코드를 보고 피드백 해주세요.
        코드에 대한 피드백은 Performance Optimization과 Readability 관점에서 두 가지 측면에서 작성해주세요.

        <code>{code}</code>

        만약 코드에 대한 더 이상의 개선의 여지가 없다면 "{NO_MORE_CODE}"라고 작성해주세요.
        """}
    ]

    response = get_messages(messages)

    return response.content[0].text


def is_stop_conditions(feedback, revision_num):
    return NO_MORE_CODE in feedback or revision_num > 3


def refine_code(code, feedback):
    messages = [
        {"role": "user", "content": f"""
        다음은 코드에 대한 피드백과 코드입니다.
        피드백을 반영해서 코드를 개선해주세요.
        개선된 코드 또한 <code> 태그 내에 작성해 주세요.

        <feedback>{feedback}</feedback>

        <code>{code}</code>
        """
         }
    ]

    response = get_messages(messages)

    return extract_code(response.content[0].text)


In [ ]:
messages = [
    {"role": "user", "content": "두 개의 정렬된 배열을 병합하고, 하나의 정렬된 배열로 만드는 함수를 작성하세요. 단, 병합은 추가적인 정렬없이 수행돼야 합니다."}
]

print(write_self_refine(messages))

In [ ]:
from anthropic import AsyncAnthropic

client = AsyncAnthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))


async def get_message_async(message, system_prompt="", tools=[]):
    return await client.messages.create(
        model="claude-3-5-sonnet-20241022",
        messages=message,
        system=system_prompt,
        tools=tools,
        max_tokens=1024 * 1,
        temperature=0,
    )

In [ ]:
import asyncio
import nest_asyncio

nest_asyncio.apply()


async def chain_of_verification(query):
    initial_answer = await get_initial_answer(query)
    print(f"initial_answer: {initial_answer}")
    print()
    verification_questions = await get_verification_questions(initial_answer, query)
    print(f"verification_questions: {verification_questions}")
    print()
    verification_questions_answers = await get_verification_answers(verification_questions)
    print(f"verification_questions_answers: {verification_questions_answers}")
    print()
    return await final_answer(verification_questions_answers, query, initial_answer)

async def get_initial_answer(query):
    messages = [
        {"role": "user", "content": query}
    ]
    response = await get_message_async(messages)
    return response.content[0].text


async def get_verification_questions(initial_answer, query):
    system_prompt = """
    당신은 "질문에 해당하는 응답이 올바른 것인지"를 검증할 질문을 만드는 뛰어난 전문 AI 시스템입니다.
    이러한 질문은 생성된 응답의 주요 가정, 사실 및 기타 중요한 부분을 검증해야합니다.
    """

    messages = [
        {"role": "user", "content": f"""
        다음 질문과 응답을 보고, 질문에 대한 응답이 적절한지 검증하기 위한 후속 질문을 생성해주세요.
        <question>{query}</question>
        <answer>{initial_answer}</answer>
        """
         }
    ]

    tools = [
        {
            "name": "verification_question",
            "description": "질문에 대한 응답을 검증하기 위한 질문 목록입니다.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "questions": {
                        "type": "array",
                        "items": {
                            "type": "string",
                            "description": "검증하기 위한 질문"
                        }
                    }
                },
                "required": ["questions"]
            },
        }
    ]
    response = await get_message_async(messages, system_prompt, tools)

    for c in response.content:
        if c.type == "tool_use" and c.name == "verification_question":
            return c.input["questions"]


async def get_verification_answers(verification_questions):
    system_prompt = "당신은 검증 질문에 답하는 데 뛰어난 AI 시스템입니다."

    async def get_single_answer(question):
        messages = [
            {"role": "user", "content": f"""
            다음 질문에 답하세요.
            question>{question}</question>
            """}
        ]

        response = await get_message_async(messages, system_prompt)
        return f"질문: {question}\n답변: {response.content[0].text}"

    tasks = [get_single_answer(q) for q in verification_questions]
    return await asyncio.gather(*tasks)


async def final_answer(verification_questions_and_answer, query, initial_answer):
    system_prompt = """
    당신은 초기 답변이 초기 질문에 적절하게 작성됐는지 확인하는 AI 시스템입니다.
    판단 근거는 검증 질문과 답변을 기반으로 판단하세요.
    만약 초기 답변이 올바르다고 판단한다면 "답변"을 반환하고, 그렇지 않을 경우 검증 질문과 답변을 기반으로 새로운 응답을 직접 작성해 반환하세요.
    """

    messages = [
        {"role": "user", "content": f"""
        <initial_questsion>{query}</initial_questsion>
        <initial_answer>{initial_answer}</initial_answer>
        <verification_questions_and_answer>{verification_questions_and_answer}</verification_questions_and_answer>
        """}
    ]

    return await get_message_async(messages, system_prompt)

In [ ]:
asyncio.run(chain_of_verification("대한민국 경상북도 구미에서 태어난 유명인들은?"))